In [1]:
#1.get the polygen coords from json
#2.get the polygen obj from image
#3.find another coords that has no iou with exited obj
#4.paste obj to image and json file
import json
import matplotlib.pyplot as plt
import numpy as np

import os
import cv2
import PIL

from labelme import utils
import annotation_all_pipline as aapl
import random
import shapely
from shapely.geometry import Polygon
from collections import Counter
import datetime

In [2]:
SEED_JASON_DIR = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/JASON_DATASETS/\
data_annotated/'
SEED_JASON_DIR = '/media/yyp/DATA/tempimgs/'
def find_target_files(target_dir, endswith):
    all_files = os.listdir(target_dir)
    targets = []
    for f in all_files:
        if(f.endswith(endswith)):
            targets.append(f)
    return targets

def get_polygen_from_obj(jsobj):
    return jsobj['points']

def get_imgobj_from_polygen(polygen):
    return ''

def get_objs_from_json(json_file, datasets_name):
    data = json.load(open(json_file))
    objs = data[datasets_name]
    return objs

def get_imageData_from_json(json_file):
    data = json.load(open(json_file))
    img = data['imageData']
    img = utils.img_b64_to_arr(img)
    return img

def plt_show_img(img):
    plt.imshow(img)
    plt.show()
    
def polygons_to_mask(img_shape, polygons):
 
    mask = np.zeros(img_shape, dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    xy = list(map(tuple, polygons))
    PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask
 
def polygons_to_mask2(img_shape, polygons):
 
    mask = np.zeros(img_shape, dtype=np.uint8)
    polygons = np.asarray([polygons], np.int32) # 这里必须是int32，其他类型使用fillPoly会报错
    # cv2.fillPoly(mask, polygons, 1) # 非int32 会报错
    cv2.fillPoly(mask, polygons, (255,255,255))  # 非int32 会报错
    return mask

def get_random_point(y_max, x_max):
    array_x = np.arange(x_max)
    array_y = np.arange(y_max)
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    return (random_x, random_y)

def caculate_move_vector(x_y_coords1, x_y_coords2):
    return (x_y_coords2[0] - x_y_coords1[0], x_y_coords2[1] - x_y_coords1[1])

def get_new_polygen(polygen, move_vector):
    new_polygen_array = np.array(polygen) + move_vector
    new_polygen_list = [list(i) for i in new_polygen_array]
    return  new_polygen_list

def get_bounding_rect(polygen):
    polygen_arry = np.array(polygen) # 必须是array数组的形式
    rect_1 = cv2.boundingRect(polygen_arry.astype(int)) #　(xmin,ymin,w,h)
    rect_2 = cv2.minAreaRect(polygen_arry.astype(int)) # 得到最小外接矩形的（中心(x,y), (宽,高), 旋转角度）
    return rect_1
def rect_2_bbox(rect):
    return [rect[0], int(rect[0]+rect[2]), 
            rect[1], int(rect[1]+rect[3])]

def get_polygens_from_json_objs(objs):
    return [o['points'] for o in objs]

def get_polygens_labels_from_json_objs(objs):
    return [o['label'] for o in objs]

def remove_seperated_obj_label(labels):
    label_dict = dict(Counter(labels))
    for k,v in label_dict.items():
        if(v > 1):
            labels = list(filter(lambda x:x!=k,labels))
    return labels

def check_new_polygen(polygens, new_polygen):
    polygen2 = Polygon(new_polygen)
    for p in polygens:
        polygen1 = Polygon(p)
        if(polygen1.intersects(polygen2)):
            return False
    return True   
def to_json_file(loaded_dict_data, file):
    with open(file, "w") as f:
        json.dump(loaded_dict_data, f)
        print("[ INFO ] write success !")
        
def get_empty_coco_format_json_dict_template():
    coco_json_context_tamplate = {"flags": {},\
                                  "imageWidth": 1920,\
                                  "imageHeight": 1080,\
                                  "imagePath":"test.jpg",\
                                  "shapes": [{"points": [[5.0,5.0],[20.0,20.0], [15.0, 30.0]],\
                                              "shape_type": "",\
                                              "flags": {},\
                                              "label": "",\
                                              "group_id":None}],\
                                  "version": "4.2.9",\
                                  "imageData":None}
    return coco_json_context_tamplate

def get_empty_coco_format_obj_dict_template():
    obj_dict = {"points": [[5.0,5.0],[20.0,20.0], [15.0, 30.0]],\
                "shape_type": "polygon",\
                "flags": {},\
                "label": "",\
                "group_id":None
               }
    return obj_dict

def save_dict_to_json_file(jsonDict, fileName):
    with open(fileName, "w", encoding='utf-8') as f:
        f.write(json.dumps(jsonDict, indent=2))
        
jsons = find_target_files(SEED_JASON_DIR, '.json')
jpgs = find_target_files(SEED_JASON_DIR, '.jpg')
jsons.sort()
jpgs.sort()
print(len(jpgs))
print(len(jsons))

80
16


In [3]:
LOOP_TIMES = 1
for i in range(LOOP_TIMES):
    for j in jsons:
        data = json.load(open(SEED_JASON_DIR + j))
        #bs64img = data['imageData']
        #img = utils.img_b64_to_arr(bs64img)
        #img = img[:,:,(2,1,0)]
        print(SEED_JASON_DIR + j[:-5] + '.jpg')
        img = cv2.imread(SEED_JASON_DIR + j[:-5] + '.jpg')
        src = img.copy()
        print(img.shape)

        objs = get_objs_from_json(SEED_JASON_DIR + j, 'shapes')
        objs_extend = objs.copy()
        i = 0
        polygens = get_polygens_from_json_objs(objs)
        labels = get_polygens_labels_from_json_objs(objs)
        #labels = remove_seperated_obj_label(labels)
        for o in objs: #o is a dict
            o_extend = o.copy()
            label = o['label']
            os.system('mkdir -p ' + SEED_JASON_DIR + label)
            polygen = o['points']

            rect_src = get_bounding_rect(polygen)
            bbox_old = rect_2_bbox(rect_src)
            
            time_stamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")
            mask = polygons_to_mask2(img.shape, polygen)
            mask_obj = mask[bbox_old[2]:bbox_old[3], bbox_old[0]:bbox_old[1],:]
            src_obj = img[bbox_old[2]:bbox_old[3], bbox_old[0]:bbox_old[1],:]
            cv2.imwrite(SEED_JASON_DIR + label + '/' + j[:-5] + '_' + str(i) + '_msk.jpg', mask_obj)
            cv2.imwrite(SEED_JASON_DIR + label + '/' + j[:-5] + '_' + str(i) + '_obj.jpg', src_obj)
            
            move_vector = (-rect_src[0], -rect_src[1])
            coco_format_obj_dict = get_empty_coco_format_obj_dict_template()
            coco_format_obj_dict['points'] = get_new_polygen(polygen, move_vector)
            coco_format_obj_dict['label'] = label
            
            coco_format_json_dict = get_empty_coco_format_json_dict_template()
            coco_format_json_dict['imageWidth'] = rect_src[2]
            coco_format_json_dict['imageHeight'] = rect_src[3]
            coco_format_json_dict['imagePath'] = SEED_JASON_DIR + label + '/' + j[:-5] + '_' + str(i) + '_obj.jpg'
            coco_format_json_dict['shapes'] = [coco_format_obj_dict]
            save_dict_to_json_file(coco_format_json_dict, SEED_JASON_DIR + label + '/' + j[:-5] + '_' + str(i) + '_obj.json')
            
            i += 1
        #break

/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17236.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17237.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17240.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17245.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17255.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17256.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17257.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17259.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17260.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17265.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17270.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17274.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17281.jpg
(756, 1352, 3)
/media/yyp/DATA/tempimgs/kk_2020-08-09_17-02-17291.jpg
(756, 1352, 3)
/media/yyp/DATA/temp